In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [57]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import regex as re
import pickle
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import BaggingClassifier, StackingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

### Step 1: Data Loading

In [58]:
df_train=pd.read_csv('/content/train (1).csv')
df_train

,MachineID,ProductName,EngineVersion,AppVersion,SignatureVersion,IsBetaUser,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,...,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,IsGamer,RegionIdentifier,DateAS,DateOS,target
0,f541bae429089117c4aac39c90dd3416,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1003.0,0,7.0,0,53447.0,1.0,...,0,0.0,1,0,1.0,0.0,6.0,2018-09-10 10:11:00,2018-04-17,0
1,dc2b14d9ce3a0ce4050bb640190f2ca5,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1465.0,0,7.0,0,53447.0,1.0,...,1,0.0,0,0,0.0,0.0,10.0,2018-08-16 00:01:00,2018-08-14,1
2,fd20c5f010e9c5f91ad1c6b3e0da68a0,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1546.0,0,7.0,0,53447.0,1.0,...,0,0.0,0,0,0.0,1.0,6.0,2018-09-20 23:20:00,2018-09-11,1
3,38711eae85eb77a72ec5dfdf27eb2a76,win8defender,1.1.15200.1,4.12.17007.18011,1.275.1141.0,0,7.0,0,46413.0,2.0,...,1,0.0,0,0,0.0,0.0,12.0,2018-09-14 00:32:00,2018-01-03,1
4,32607c9a543a9214e2c7e45800ed4849,win8defender,1.1.15200.1,4.13.17134.228,1.275.1283.0,0,7.0,0,40466.0,2.0,...,0,0.0,0,0,0.0,1.0,7.0,2018-09-15 19:34:00,2018-09-11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,a378b2196a7a1d8ee0d4f989ee46730e,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1582.0,0,7.0,0,68585.0,2.0,...,0,0.0,0,0,0.0,1.0,7.0,2018-09-21 00:59:00,2018-09-20,0
99996,7f4433ca05d24b9bbdd3932f8b05e61f,win8defender,1.1.15100.1,4.18.1806.18062,1.273.1156.0,0,7.0,0,11280.0,2.0,...,0,0.0,0,0,0.0,0.0,3.0,2018-08-10 07:23:00,2018-07-10,0
99997,d6852dfff534e03d5ddfd3c7875ab072,win8defender,1.1.15200.1,4.18.1807.18075,1.275.209.0,0,7.0,0,53447.0,1.0,...,1,0.0,0,0,0.0,1.0,1.0,2018-08-26 23:51:00,2018-03-13,1
99998,70a8dc682128c9ff78a3de5aea9dac74,win8defender,1.1.14901.4,4.16.17656.18052,1.269.641.0,0,7.0,0,53447.0,1.0,...,1,0.0,1,1,1.0,0.0,1.0,2018-06-04 17:13:00,2018-05-08,0


In [59]:
df_test=pd.read_csv("/content/test (1).csv")
df_test

,MachineID,ProductName,EngineVersion,AppVersion,SignatureVersion,IsBetaUser,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,...,FirmwareVersionID,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,IsGamer,RegionIdentifier,DateAS,DateOS
0,c2c61173092ad20db5585ca8c10de6e0,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1276.0,0,7.0,0,53447.0,1.0,...,48751.0,0,0.0,0,0,0.0,1.0,11.0,2018-08-12 13:12:00,2018-07-10
1,2e6bb5362ba1ce897feeb1de5edea686,win8defender,1.1.15100.1,4.10.209.0,1.273.371.0,0,7.0,0,46184.0,2.0,...,64709.0,1,0.0,0,0,0.0,0.0,12.0,2018-07-26 06:54:00,2016-10-11
2,692649baed1c50bb8d051ae7f06e3625,win8defender,1.1.15200.1,4.18.1807.18075,1.275.263.0,0,7.0,0,51954.0,2.0,...,33094.0,1,0.0,0,0,0.0,0.0,10.0,2018-08-27 17:29:00,2018-05-08
3,2ff95e7d97e9f80ec81c97622dd93ed6,win8defender,1.1.15300.6,4.18.1807.18075,1.277.58.0,0,7.0,0,41571.0,2.0,...,26184.0,1,0.0,0,0,0.0,0.0,10.0,2018-09-25 22:13:00,2018-05-08
4,256766b4c5011e146f05b323863e08a4,win8defender,1.1.14800.3,4.14.17639.18041,1.267.898.0,0,7.0,0,40645.0,3.0,...,12126.0,0,0.0,0,0,0.0,0.0,7.0,2018-05-06 01:31:00,2018-06-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,42d3ddb25eeb149517e2b6871b061df4,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1329.0,0,7.0,0,32554.0,3.0,...,20107.0,0,0.0,0,0,0.0,0.0,13.0,2018-09-16 18:15:00,2018-09-11
9996,3d474c494d797cafc2ab1665711f0f91,win8defender,1.1.15100.1,4.11.15063.0,1.273.894.0,0,7.0,0,53447.0,1.0,...,11386.0,0,0.0,0,0,0.0,1.0,1.0,2018-08-05 22:50:00,2017-04-05
9997,9aee2fc73af8240a688ceb83d41c0418,win8defender,1.1.15200.1,4.18.1807.18075,1.275.504.0,0,7.0,0,47238.0,2.0,...,12247.0,1,0.0,0,0,0.0,0.0,11.0,2018-08-31 16:03:00,2018-06-12
9998,a2f8c1f88b78271c38767fe43850b359,win8defender,1.1.15200.1,4.18.1807.18075,1.275.461.0,0,7.0,0,53447.0,1.0,...,4269.0,1,0.0,0,0,0.0,0.0,10.0,2018-08-30 18:35:00,2017-12-12


In [60]:
df_train.shape

(100000, 76)

In [50]:
df_test.shape

(10000, 75)

#### Conclusion: Data is loaded from CSV files for further processing

### **Step 2:** **Exploratory Data Analysis (EDA)**

In [61]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 76 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   MachineID                           100000 non-null  object 
 1   ProductName                         100000 non-null  object 
 2   EngineVersion                       100000 non-null  object 
 3   AppVersion                          100000 non-null  object 
 4   SignatureVersion                    100000 non-null  object 
 5   IsBetaUser                          100000 non-null  int64  
 6   RealTimeProtectionState             99934 non-null   float64
 7   IsPassiveModeEnabled                100000 non-null  int64  
 8   AntivirusConfigID                   99924 non-null   float64
 9   NumAntivirusProductsInstalled       99924 non-null   float64
 10  NumAntivirusProductsEnabled         99924 non-null   float64
 11  HasTpm                     

In [62]:
df_train.describe()

,IsBetaUser,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,NumAntivirusProductsEnabled,HasTpm,CountryID,CityID,GeoRegionID,...,FirmwareManufacturerID,FirmwareVersionID,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,IsGamer,RegionIdentifier,target
count,100000.0,99934.000000,100000.000000,99924.000000,99924.000000,99924.000000,100000.000000,100000.000000,99377.000000,100000.000000,...,99624.000000,99666.000000,100000.000000,99980.000000,100000.000000,100000.000000,99866.000000,99441.000000,99441.000000,100000.000000
mean,0.0,6.848430,0.017620,47975.710440,1.326528,1.018264,0.996780,108.078790,81029.938587,169.741630,...,401.987613,32942.648044,0.495690,0.003841,0.128470,0.040580,0.058398,0.296668,7.875866,0.505250
std,0.0,1.015166,0.131566,13803.321533,0.520681,0.155291,0.056654,63.062151,48944.027074,89.188929,...,221.318891,21151.970827,0.499984,0.061855,0.334614,0.197316,0.234496,0.456791,4.562533,0.499975
min,0.0,0.000000,0.000000,39.000000,1.000000,0.000000,0.000000,1.000000,7.000000,1.000000,...,2.000000,121.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.0,7.000000,0.000000,49480.000000,1.000000,1.000000,1.000000,51.000000,36694.000000,89.000000,...,142.000000,13020.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,0.0,7.000000,0.000000,53447.000000,1.000000,1.000000,1.000000,97.000000,82373.000000,181.000000,...,500.000000,33066.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000
75%,0.0,7.000000,0.000000,53447.000000,2.000000,1.000000,1.000000,162.000000,122835.000000,267.000000,...,556.000000,52357.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,11.000000,1.000000
max,0.0,8.000000,1.000000,70490.000000,5.000000,4.000000,1.000000,222.000000,167957.000000,296.000000,...,1083.000000,72091.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000,1.000000


In [63]:
df_train.columns

Index(['MachineID', 'ProductName', 'EngineVersion', 'AppVersion',
       'SignatureVersion', 'IsBetaUser', 'RealTimeProtectionState',
       'IsPassiveModeEnabled', 'AntivirusConfigID',
       'NumAntivirusProductsInstalled', 'NumAntivirusProductsEnabled',
       'HasTpm', 'CountryID', 'CityID', 'GeoRegionID', 'LocaleEnglishNameID',
       'PlatformType', 'Processor', 'OSVersion', 'OSBuildNumber',
       'OSProductSuite', 'OsPlatformSubRelease', 'OSBuildLab',
       'SKUEditionName', 'IsSystemProtected', 'AutoSampleSubmissionEnabled',
       'SMode', 'IEVersionID', 'FirewallEnabled', 'EnableLUA',
       'MDC2FormFactor', 'DeviceFamily', 'OEMNameID', 'OEMModelID',
       'ProcessorCoreCount', 'ProcessorManufacturerID', 'ProcessorModelID',
       'PrimaryDiskCapacityMB', 'PrimaryDiskType', 'SystemVolumeCapacityMB',
       'HasOpticalDiskDrive', 'TotalPhysicalRAMMB', 'ChassisType',
       'PrimaryDisplayDiagonalInches', 'PrimaryDisplayResolutionHorizontal',
       'PrimaryDisplayResolutio

In [64]:
df_train.count()

,0
MachineID,100000
ProductName,100000
EngineVersion,100000
AppVersion,100000
SignatureVersion,100000
...,...
IsGamer,99441
RegionIdentifier,99441
DateAS,100000
DateOS,99976


In [65]:
df_train .isnull().sum()

,0
MachineID,0
ProductName,0
EngineVersion,0
AppVersion,0
SignatureVersion,0
...,...
IsGamer,559
RegionIdentifier,559
DateAS,0
DateOS,24


In [66]:
# sns.pairplot(df_train)
# plt.show()

#### Conclusion: EDA helps understand the dataset's structure, distributions, and relationships.

### **Step 3: Data Preprocessing**

In [67]:
target_column = 'target'  # Replace with actual target column name
X = df_train.drop(columns=[target_column])
y = df_train[target_column]

#### **Handle categorical variables**

In [68]:
categorical_cols = X.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

#### **Scale numerical data**

In [69]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

#### **Train-test split**

In [70]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
print("Data preprocessing completed.")

Data preprocessing completed.


#### # Conclusion: Categorical variables are encoded, numerical features are scaled, and data is split.

### **Step 4: Model Training (Random Forest, XGBoost, LightGBM)**
#### **Random Forest**

In [71]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_valid)
print("Random Forest Accuracy:", accuracy_score(y_valid, y_pred_rf))
print(classification_report(y_valid, y_pred_rf))

Random Forest Accuracy: 0.6169
              precision    recall  f1-score   support

           0       0.62      0.60      0.61      9878
           1       0.62      0.64      0.63     10122

    accuracy                           0.62     20000
   macro avg       0.62      0.62      0.62     20000
weighted avg       0.62      0.62      0.62     20000



####  Conclusion: Random Forest is an ensemble method that helps reduce overfitting.

### XGBoost

In [72]:
xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_valid)
print("XGBoost Accuracy:", accuracy_score(y_valid, y_pred_xgb))
print(classification_report(y_valid, y_pred_xgb))

XGBoost Accuracy: 0.6291
              precision    recall  f1-score   support

           0       0.64      0.58      0.61      9878
           1       0.62      0.68      0.65     10122

    accuracy                           0.63     20000
   macro avg       0.63      0.63      0.63     20000
weighted avg       0.63      0.63      0.63     20000



#### # Conclusion: XGBoost is an optimized gradient boosting method that improves accuracy.

#### **LightGBM**

In [73]:
lgb_model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_valid)
print("LightGBM Accuracy:", accuracy_score(y_valid, y_pred_lgb))
print(classification_report(y_valid, y_pred_lgb))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 40403, number of negative: 39597
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.098884 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4963
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 73
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.505038 -> initscore=0.020151
[LightGBM] [Info] Start training from score 0.020151


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LightGBM Accuracy: 0.62985
              precision    recall  f1-score   support

           0       0.64      0.59      0.61      9878
           1       0.62      0.67      0.65     10122

    accuracy                           0.63     20000
   macro avg       0.63      0.63      0.63     20000
weighted avg       0.63      0.63      0.63     20000



####  Conclusion: LightGBM is an efficient boosting algorithm optimized for large datasets.


### **Step 5: Save Best Model**

In [74]:
best_model = max([(rf_model, accuracy_score(y_valid, y_pred_rf)),
                  (xgb_model, accuracy_score(y_valid, y_pred_xgb)),
                  (lgb_model, accuracy_score(y_valid, y_pred_lgb))], key=lambda x: x[1])[0]

In [31]:
df=pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
x=df.drop("target",axis=1)
y=df["target"]

from sklearn.dummy import DummyClassifier
model = DummyClassifier().fit(x,y)

x_test = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")
y_pred = model.predict(x_test)

submission = pd.DataFrame({"id": range(0,x_test.shape[0]),"target": y_pred})

submission.to_csv("submission.csv",index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/System-Threat-Forecaster/train.csv'

In [ ]:
# 1.How many unique versions of the operating system are present in the dataset?
unique_os_version= df_train['OSVersion'].unique()
print("Number of unique OS versions:", unique_os_version)

In [ ]:
#2)What is the maximum value of the feature “NumAntivirusProductsInstalled”?
max=df_train['NumAntivirusProductsInstalled'].max()
max

In [ ]:
#3)In how many systems owned by gamers was malware detected?
malware_detected=df_train[(df_train['IsGamer']==1)].shape[0]
malware_detected

In [ ]:
#4)For observations where the feature “IsPassiveModeEnabled” equals “1,” what is the most frequent value of the feature “RealTimeProtectionState”?
# Filter the dataset where IsPassiveModeEnabled == 1
filtered_data = df[df["IsPassiveModeEnabled"] == 1]
filtered_data

# Find the most frequent value of RealTimeProtectionState
most_frequent_value = filtered_data["RealTimeProtectionState"].mode()[0]

most_frequent_value

In [ ]:
#5) How many systems have a screen resolution of 1366 x 768?
num_systems = df[(df["PrimaryDisplayResolutionHorizontal"] == 1366) &
                 (df["PrimaryDisplayResolutionVertical"] == 768)].shape[0]

num_systems


In [ ]:
# What is the 50th percentile value of “TotalPhysicalRAMMB”?
k=np.percentile(df_train['TotalPhysicalRAMMB'],50)
k

In [ ]:
df_train.describe()

In [ ]:
#Explain the Data using any Exploratory Data Analysis (EDA) you have used
#EDA helps us understand the structure, distribution, and patterns in the dataset.

# Exploratory Data Analysis (EDA)
## 1) understanding the dataset
### check dataset shape(row & col)

In [ ]:
df_train.shape

In [ ]:
# Preview the first few rows
df.head()

In [ ]:
# Check column names and data types
df_train.info()

## 2)  Handling Missing Values

In [ ]:
# Count missing values per column
df_train.isnull().sum()

In [ ]:
# Percentage of missing values
df_train.isnull().mean() * 100

In [ ]:
# handling the missing values
df=df_train.dropna().sum()
df

In [ ]:
# missing vale filling
df_filled = df_train.fillna(0)  # Replace NaN with 0
df_filled
df_filled = (df_train.fillna("Unknown")) # Replace NaN with a string
df_filled

## 3. Summary Statistics


In [ ]:
#numerical feature
df_train.describe()

In [ ]:
# Categorical features
df_train.nunique()

In [ ]:
# Value counts of key categorical variables
df_train["OSVersion"].value_counts()
df_train["IsGamer"].value_counts()

## 4. Distribution Analysis

In [ ]:
# histogram for numerical features
import matplotlib.pyplot as plt
df_train["TotalPhysicalRAMMB"].hist(bins=100)
plt.title("Distribution of RAM")
plt.xlabel("TotalPhysicalRAMMB")
plt.ylabel("Count")
plt.show()

In [ ]:
# Boxplot to identify outliers
import seaborn as sns
sns.boxplot(x=df_train["TotalPhysicalRAMMB"])
plt.show()

## 5. Categorical Feature Analysis

In [ ]:
# Bar plot for OS version
df_train["OSVersion"].value_counts().plot(kind="bar")
plt.show()

In [ ]:
# Pie chart for gamer vs non-gamer
df_train["IsGamer"].value_counts().plot(kind="pie", autopct="%1.1f%%",color=['green','red'])
plt.show()

## 6. Correlation Analysis
